In [37]:
from graphics import * 
import pandas as pd 
import numpy as np 
from new_data_loader import bench, squat, deadlift
from data_loader import be, sq, de
import plotly.graph_objects as go
import matplotlib.pyplot as plt 
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt 

interval = 100
k = 5
t = squat[0]['gravityX'].dropna().to_numpy()
bench.extend(be)
squat.extend(sq)
deadlift.extend(de)

bench = [e for e in bench if len(e['gravityX'].dropna().to_numpy()) > interval]
squat = [e for e in squat if len(e['gravityX'].dropna().to_numpy()) > interval]
deadlift = [e for e in deadlift if len(e['gravityX'].dropna().to_numpy()) > interval]

In [78]:
def process_signal(sig, interval, normalize=False):
    if normalize: 
        mean = np.mean(sig)
        std = np.std(sig)
        sig = (sig - mean)/std
    if interval not in sig.shape:
        sig = sig.reshape(-1, 1)
    pad_len = interval - (sig.shape[0] % interval)
    if pad_len != interval:
        padding = np.zeros((pad_len, sig.shape[1]))
        padding[:, :] = sig[-1, -1]
        sig = np.concatenate([sig, padding], axis=0)
    sig = sig.reshape(-1, interval, sig.shape[1])
    sig = np.squeeze(sig)
    return sig

def get_mean_std_tensor(sig, interval, normalize=False):
    temp = process_signal(sig, interval, normalize)
    tensor = np.vstack([np.average(temp, axis=0), np.std(temp, axis=0)])
    tensor = np.average(tensor, axis=1)
    return tensor

In [54]:
train_size = 5
labels = ['gravityX', 'gravityY', 'gravityZ']

In [55]:
train_bench = bench[:train_size]
train_squat = squat[:train_size]
train_deadlift = deadlift[:train_size]

train_X_bench = [np.array([get_mean_std_tensor(e[l].dropna().to_numpy(), interval) for l in labels]).T for e in train_bench]
train_X_squat = [np.array([get_mean_std_tensor(e[l].dropna().to_numpy(), interval) for l in labels]).T for e in train_squat]
train_X_deadlift = [np.array([get_mean_std_tensor(e[l].dropna().to_numpy(), interval) for l in labels]).T for e in train_deadlift]

train_X = np.array(train_X_bench + train_X_squat + train_X_deadlift)


train_y = ['bench' for e in train_bench]
train_y += ['squat' for e in train_squat]
train_y += ['deadlift' for e in train_deadlift]
train_y = np.array(train_y)

In [56]:
np.array(train_X).shape

(15, 2, 3)

In [57]:
test_bench = bench[train_size:]
test_squat = squat[train_size:]
test_deadlift = deadlift[train_size:]

test_X_bench = [np.array([get_mean_std_tensor(e[l].dropna().to_numpy(), interval) for l in labels]).T for e in test_bench]
test_X_squat = [np.array([get_mean_std_tensor(e[l].dropna().to_numpy(), interval) for l in labels]).T for e in test_squat]
test_X_deadlift = [np.array([get_mean_std_tensor(e[l].dropna().to_numpy(), interval) for l in labels]).T for e in test_deadlift]

test_X = np.array(test_X_bench + test_X_squat + test_X_deadlift)

test_y = ['bench' for e in test_bench]
test_y += ['squat' for e in test_squat]
test_y += ['deadlift' for e in test_deadlift]
test_y = np.array(test_y)

In [58]:
dist = [np.sum((e - test_X[0])**2) for e in train_X]
min_idx = np.argpartition(dist, k)[:k]
unique, counts = np.unique(train_y[min_idx], return_counts=True)
res = unique[counts == counts.max()]
print("Most frequent numbers are : " + str(res) + " with count " + str(counts.max()) + ' Actual: ' + train_y[0])

Most frequent numbers are : ['bench'] with count 4 Actual: bench


In [59]:
total = 0
acc = 0
for t, label in zip(test_X, test_y):
    dist = [np.sum((e - t)**2) for e in train_X]
    min_idx = np.argpartition(dist, k)[:k]
    unique, counts = np.unique(train_y[min_idx], return_counts=True)
    res = unique[counts == counts.max()]
    print("Most frequent numbers are : " + str(res) + " with count " + str(counts.max()) + ' Actual: ' + label)
    if res[0] == label: 
        acc += 1

    total += 1

acc/total

Most frequent numbers are : ['bench'] with count 4 Actual: bench
Most frequent numbers are : ['bench'] with count 4 Actual: bench
Most frequent numbers are : ['bench' 'squat'] with count 2 Actual: bench
Most frequent numbers are : ['bench'] with count 4 Actual: bench
Most frequent numbers are : ['bench'] with count 3 Actual: bench
Most frequent numbers are : ['bench'] with count 3 Actual: bench
Most frequent numbers are : ['bench'] with count 3 Actual: bench
Most frequent numbers are : ['bench'] with count 3 Actual: bench
Most frequent numbers are : ['bench'] with count 3 Actual: bench
Most frequent numbers are : ['bench'] with count 4 Actual: bench
Most frequent numbers are : ['bench'] with count 3 Actual: bench
Most frequent numbers are : ['bench'] with count 3 Actual: bench
Most frequent numbers are : ['bench'] with count 3 Actual: bench
Most frequent numbers are : ['bench' 'deadlift'] with count 2 Actual: bench
Most frequent numbers are : ['bench'] with count 4 Actual: bench
Most f

0.8248175182481752

In [62]:

total = 0
acc = 0
for t, label in zip(test_X, test_y):
    dist = [np.sum((e - t)**2) for e in train_X]
    min_idx = np.argpartition(dist, k)[:k]
    unique, counts = np.unique(train_y[min_idx], return_counts=True)
    res = unique[counts == counts.max()]
    print("Most frequent numbers are : " + str(res) + " with count " + str(counts.max()) + ' Actual: ' + label)
    if res[0] == label: 
        acc += 1

    total += 1

acc/total

Most frequent numbers are : ['bench'] with count 4 Actual: bench
Most frequent numbers are : ['bench'] with count 4 Actual: bench
Most frequent numbers are : ['bench' 'squat'] with count 2 Actual: bench
Most frequent numbers are : ['bench'] with count 4 Actual: bench
Most frequent numbers are : ['bench'] with count 3 Actual: bench
Most frequent numbers are : ['bench'] with count 3 Actual: bench
Most frequent numbers are : ['bench'] with count 3 Actual: bench
Most frequent numbers are : ['bench'] with count 3 Actual: bench
Most frequent numbers are : ['bench'] with count 3 Actual: bench
Most frequent numbers are : ['bench'] with count 4 Actual: bench
Most frequent numbers are : ['bench'] with count 3 Actual: bench
Most frequent numbers are : ['bench'] with count 3 Actual: bench
Most frequent numbers are : ['bench'] with count 3 Actual: bench
Most frequent numbers are : ['bench' 'deadlift'] with count 2 Actual: bench
Most frequent numbers are : ['bench'] with count 4 Actual: bench
Most f

0.8248175182481752

In [ ]:
def eye_diag(sig, interval):

    fig = go.Figure()
    print(sig.shape)
    for e in sig:
        fig.add_trace(
            go.Scatter(x=list(range(len(e))), y=e)
        )
    
    fig.update_layout(
        height=600, 
        title="Gravity Signals Over Time",
        xaxis_title="Time Steps",
        hovermode="x unified", 
    )
    
    fig.show()



In [67]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter

# Parameters
segment_len = 100
axes = ['gravityX', 'gravityY', 'gravityZ']

def get_segments(df, label):
    segments = []
    labels = []
    min_len = min([len(df[axis].dropna()) for axis in axes])
    max_chunks = min_len // segment_len

    for i in range(max_chunks):
        chunk = []
        for axis in axes:
            vals = df[axis].dropna().values[i*segment_len : (i+1)*segment_len]
            chunk.append(vals)
        segment = np.concatenate(chunk)
        segments.append(segment)
        labels.append(label)
    return segments, labels

# Gather all segments
X, y = [], []
for df in train_bench:
    segs, labels = get_segments(df, 'bench')
    X.extend(segs)
    y.extend(labels)

for df in train_squat:
    segs, labels = get_segments(df, 'squat')
    X.extend(segs)
    y.extend(labels)

for df in train_deadlift:
    segs, labels = get_segments(df, 'deadlift')
    X.extend(segs)
    y.extend(labels)

X = np.array(X)
y = np.array(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.95, random_state=42)

# Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

       bench       1.00      1.00      1.00        61
    deadlift       1.00      0.98      0.99       101
       squat       0.97      1.00      0.99        68

    accuracy                           0.99       230
   macro avg       0.99      0.99      0.99       230
weighted avg       0.99      0.99      0.99       230



In [80]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter

# Parameters
segment_len = 100
axes = ['gravityX', 'gravityY', 'gravityZ']

def get_segments(df, label):
    segments = []
    labels = []
    min_len = min([len(df[axis].dropna()) for axis in axes])
    max_chunks = min_len // segment_len

    for i in range(max_chunks):
        chunk = []
        for axis in axes:
            vals = df[axis].dropna().values[i*segment_len : (i+1)*segment_len]
            mean = np.mean(vals)
            std = np.std(vals)
            vals = (vals - mean)/std

            chunk.append(vals)
        segment = np.concatenate(chunk)
        segments.append(segment)
        labels.append(label)
    return segments, labels

# Gather all segments
X, y = [], []
for df in train_bench:
    segs, labels = get_segments(df, 'bench')
    X.extend(segs)
    y.extend(labels)

for df in train_squat:
    segs, labels = get_segments(df, 'squat')
    X.extend(segs)
    y.extend(labels)

for df in train_deadlift:
    segs, labels = get_segments(df, 'deadlift')
    X.extend(segs)
    y.extend(labels)

X = np.array(X)
y = np.array(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.95, random_state=42)

# Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

       bench       0.80      0.07      0.12        61
    deadlift       0.39      0.28      0.32       101
       squat       0.32      0.72      0.44        68

    accuracy                           0.35       230
   macro avg       0.50      0.35      0.30       230
weighted avg       0.48      0.35      0.31       230



In [ ]:
import tensorflow as tf
import numpy as np

# Load or define your model
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu', input_shape=(min_length, 3)),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Save model
model.save("cnn_model")

# Representative dataset function
def representative_data_gen():
    for _ in range(100):
        # Random dummy input. Replace with actual preprocessed data.
        data = np.random.rand(1, min_length, 3).astype(np.float32)
        yield [data]

# Convert model to TFLite with int8 quantization
converter = tf.lite.TFLiteConverter.from_saved_model("cnn_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # <- make sure inputs are int8
converter.inference_output_type = tf.int8  # <- make sure outputs are int8

tflite_quant_model = converter.convert()

# Save TFLite model
with open("cnn_model_int8.tflite", "wb") as f:
    f.write(tflite_quant_model)
